In [134]:
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
from keras import backend as K
from keras import applications
import os
import numpy as np
from keras import optimizers

In [29]:
train_path = 'D:\\training\\dogvscat\\train'
validation_path = 'D:\\training\\dogvscat\\validation'

In [24]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [28]:

img = load_img(cat_path+'\\cat.0.jpg')
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

## Training our own model

In [35]:
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 5
batch_size = 16

# dimensions of our images.
img_width, img_height = 150, 150

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 20002 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.
Epoch 1/5
125/125 [==============================] - 85s 681ms/step - loss: 0.7002 - acc: 0.5315 - val_loss: 0.6797 - val_acc: 0.6000
Epoch 2/5
125/125 [==============================] - 82s 655ms/step - loss: 0.6887 - acc: 0.5665 - val_loss: 0.6584 - val_acc: 0.5887
Epoch 3/5
125/125 [==============================] - 83s 663ms/step - loss: 0.6400 - acc: 0.6265 - val_loss: 0.5900 - val_acc: 0.7087
Epoch 4/5
125/125 [==============================] - 89s 709ms/step - loss: 0.6332 - acc: 0.6515 - val_loss: 0.5900 - val_acc: 0.7125
Epoch 5/5
125/125 [==============================] - 84s 671ms/step - loss: 0.6294 - acc: 0.6535 - val_loss: 0.5659 - val_acc: 0.7462


## Using Bottleneck feature of pretrained VGG16 model

In [80]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 5
batch_size = 16

In [47]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. /255)
    
    model = applications.VGG16(include_top=False, weights='imagenet')
    
    #train data generator
    generator = datagen.flow_from_directory(
        train_path,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = None,
        shuffle = False
    )
    
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size
    )
    
    np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
    
    #validation data generator
    generator = datagen.flow_from_directory(
        validation_path,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = None,
        shuffle=False
    )
    
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples//batch_size
    )
    
    np.save(open('bottleneck_features_validation.npy', 'wb'), 
           bottleneck_features_validation)
    


In [48]:
save_bottleneck_features()

Found 20002 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [82]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array([0]*(nb_train_samples//2) + [1]*(nb_train_samples//2))

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array([0]*(nb_validation_samples//2) + [1]*(nb_validation_samples//2))
    
    model = Sequential()
    model.add(Flatten(input_shape = train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size,
             validation_data=(validation_data, validation_labels))
    
    model.save_weights(top_model_weights_path)

In [83]:
train_top_model()

Train on 2000 samples, validate on 800 samples
Epoch 1/5
2000/2000 [==============================] - 8s 4ms/step - loss: 0.9939 - acc: 0.4885 - val_loss: 0.7004 - val_acc: 0.5000
Epoch 2/5
2000/2000 [==============================] - 7s 3ms/step - loss: 0.7119 - acc: 0.5150 - val_loss: 0.6923 - val_acc: 0.5188
Epoch 3/5
2000/2000 [==============================] - 7s 3ms/step - loss: 0.7168 - acc: 0.5385 - val_loss: 0.7119 - val_acc: 0.5088
Epoch 4/5
2000/2000 [==============================] - 6s 3ms/step - loss: 0.6819 - acc: 0.5830 - val_loss: 0.7007 - val_acc: 0.5225
Epoch 5/5
2000/2000 [==============================] - 7s 4ms/step - loss: 0.6623 - acc: 0.6145 - val_loss: 0.7058 - val_acc: 0.5088


## Fine-tuning the top layers of a a pre-trained network

In [116]:
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 5
batch_size = 16


In [117]:
input_tensor = Input(shape=(150,150,3))
#loading vgg16 model
model = applications.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)
print('Model loaded.')


Model loaded.


In [118]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [119]:
# loading the weights of previously trained top model, so that weights don't initialize randomly and cause drastic gradient changes to 
top_model.load_weights(top_model_weights_path)

In [120]:
# add the model on top of the convolutional base
model = Model(input= model.input, output= top_model(model.output))

C:\Users\mishra_s\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  


In [122]:
# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False


In [132]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0   

In [135]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])


In [137]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 20002 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [138]:
# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

C:\Users\mishra_s\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
C:\Users\mishra_s\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=5, validation_data=<keras_pre..., steps_per_epoch=125, validation_steps=800)`
  import sys
Exception ignored in: <bound method ScopedTFStatus.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFStatus object at 0x00000209066AF2E8>>
Traceback (most recent call last):
  File "C:\

Epoch 1/5
125/125 [==============================] - 2714s 22s/step - loss: 0.6697 - acc: 0.5885 - val_loss: 0.6131 - val_acc: 0.6814
Epoch 2/5
125/125 [==============================] - 3194s 26s/step - loss: 0.6069 - acc: 0.6755 - val_loss: 0.5301 - val_acc: 0.7702
Epoch 3/5
125/125 [==============================] - 3461s 28s/step - loss: 0.5136 - acc: 0.7560 - val_loss: 0.4350 - val_acc: 0.8202
Epoch 4/5
125/125 [==============================] - 3506s 28s/step - loss: 0.4642 - acc: 0.7870 - val_loss: 0.3993 - val_acc: 0.8245
Epoch 5/5
125/125 [==============================] - 2703s 22s/step - loss: 0.4196 - acc: 0.8210 - val_loss: 0.3346 - val_acc: 0.8676
